In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itard"
fs_method, fs_ratio = "chi2", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9165 | AUPR: 0.6568 | Acc: 95.44%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9193 | AUPR: 0.6656 | Acc: 95.38%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9218 | AUPR: 0.6747 | Acc: 95.33%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9160 | AUPR: 0.6566 | Acc: 95.30%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9188 | AUPR: 0.6661 | Acc: 95.33%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9209 | AUPR: 0.6734 | Acc: 95.43%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9218 | AUPR: 0.6747 | Acc: 95.33%
Internal test: AUC: 0.9255 | AUPR: 0.6990 | Acc: 95.33%
External test: AUC: 0.9241 | AUPR: 0.6742 | Acc: 95.20%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9165 | AUPR: 0.6334 | Acc: 95.07%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9162 | AUPR: 0.6297 | Acc: 94.98%
model(kernel=linear, C=1)
Valid:         AUC: 0.9157 | AUPR: 0.6252 | Acc: 94.96%
best model(kernel=linear, C=0.01)
Valid:         AUC: 0.9165 | AUPR: 0.6334 | Acc: 95.07%
Internal test: AUC: 0.9228 | AUPR: 0.6681 | Acc: 95.06%
External test: AUC: 0.9185 | AUPR: 0.6184 | Acc: 94.65%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7515 | AUPR: 0.4246 | Acc: 94.74%
model(n_neighbors=5)
Valid:         AUC: 0.7790 | AUPR: 0.4693 | Acc: 95.03%
model(n_neighbors=7)
Valid:         AUC: 0.7923 | AUPR: 0.4975 | Acc: 95.07%
best model(n_neighbors=7)
Valid:         AUC: 0.7923 | AUPR: 0.4975 | Acc: 95.07%
Internal test: AUC: 0.7993 | AUPR: 0.5025 | Acc: 94.74%
External test: AUC: 0.8047 | AUPR: 0.5008 | Acc: 94.82%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8728 | AUPR: 0.3982 | Acc: 84.44%
best model()
Valid:         AUC: 0.8728 | AUPR: 0.3982 | Acc: 84.44%
Internal test: AUC: 0.8868 | AUPR: 0.4211 | Acc: 84.89%
External test: AUC: 0.8732 | AUPR: 0.3849 | Acc: 83.42%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9165 | AUPR: 0.6522 | Acc: 95.22%
model(C=10.0)
Valid:         AUC: 0.9164 | AUPR: 0.6514 | Acc: 95.18%
model(C=100.0)
Valid:         AUC: 0.9163 | AUPR: 0.6513 | Acc: 95.16%
best model(C=1.0)
Valid:         AUC: 0.9165 | AUPR: 0.6522 | Acc: 95.22%
Internal test: AUC: 0.9243 | AUPR: 0.6894 | Acc: 95.23%
External test: AUC: 0.9168 | AUPR: 0.6387 | Acc: 94.92%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8788 | AUPR: 0.5605 | Acc: 84.35%
model(criterion=log_loss)
Valid:         AUC: 0.8723 | AUPR: 0.5755 | Acc: 83.10%
model(criterion=entropy)
Valid:         AUC: 0.8723 | AUPR: 0.5755 | Acc: 83.10%
best model(criterion=gini)
Valid:         AUC: 0.8788 | AUPR: 0.5605 | Acc: 84.35%
Internal test: AUC: 0.8689 | AUPR: 0.5973 | Acc: 83.48%
External test: AUC: 0.8651 | AUPR: 0.5516 | Acc: 82.86%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9146 | AUPR: 0.6585 | Acc: 89.98%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9154 | AUPR: 0.6584 | Acc: 90.00%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9155 | AUPR: 0.6579 | Acc: 89.96%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9140 | AUPR: 0.6582 | Acc: 89.87%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9146 | AUPR: 0.6575 | Acc: 90.02%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9145 | AUPR: 0.6576 | Acc: 89.96%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9140 | AUPR: 0.6582 | Acc: 89.87%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9146 | AUPR: 0.6575 | Acc: 90.02%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9145 | AUPR: 0.6576 | Acc: 89.96%
best model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9155 | AUPR: 0.6579 | Acc: 8

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9213 | AUPR: 0.6605 | Acc: 95.18%
model(criterion=squared_error)
Valid:         AUC: 0.9215 | AUPR: 0.6619 | Acc: 95.19%
best model(criterion=squared_error)
Valid:         AUC: 0.9215 | AUPR: 0.6619 | Acc: 95.19%
Internal test: AUC: 0.9224 | AUPR: 0.6878 | Acc: 95.26%
External test: AUC: 0.9218 | AUPR: 0.6623 | Acc: 95.06%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier


model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9176 | AUPR: 0.6534 | Acc: 95.33%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9184 | AUPR: 0.6539 | Acc: 95.33%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9194 | AUPR: 0.6569 | Acc: 95.37%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9194 | AUPR: 0.6569 | Acc: 95.37%
Internal test: AUC: 0.9193 | AUPR: 0.6715 | Acc: 94.99%
External test: AUC: 0.9142 | AUPR: 0.6279 | Acc: 94.92%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)